In [175]:
# !pip install instagrapi -U
# !pip install pyairtable
# !pip install pymorphy2 
!pip3 install nltk -U

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [177]:
import numpy as np
from instagrapi import Client
from pyairtable import Table, Api
from pyairtable.formulas import match
import nltk
import pymorphy2
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /Users/artem/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [181]:
API_KEY = 'keyAgJKFc9ssnBcue'
BASE = 'appAszMVFptj1fICD'
TABLE_NAME = 'Influencers'
THRESHOLD = 5000

nltk.download('punkt')
core = '''заповедники
экология
защита природы
алтай
кроноки
камчатка
Большой Арктический заповедник
Командорский заповедник
Остров Врангеля
Путоранский заповедник
Таймырский заповедник
Путоранская земля
УстьЛенский заповедник
Кроноцкий заповедник
Кроноцкий
Корякский
Южно-Камчатский
озера Курильское
Камбальное
Кроноцкое
Левая Щепина
Станичная
Кроноцкая
Тюшевка
Камбальная
Баргузинский заповедник
Уссурийский заповедник
Алтайский заповедник
уссурийский тигр
Алтайский заповедник
алтай
Старейший заповедник России
Кологривский лес
Ачишхо
Белогорье
Кабардино Балкарский
Эльбрус
Большой Арктический
Дагестанский заповедник
Полистовский
Убсунуровская котловина'''

core = nltk.word_tokenize(core)
set_core = set(core)

[nltk_data] Downloading package punkt to /Users/artem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [160]:
cl = Client()
cl.login('', '')

True

In [104]:
table = Table(API_KEY, BASE, TABLE_NAME)

In [105]:
VOCAB = ['заповедник', 'национальный парк', 'нацпарк', 'заказник', 'сафари']

def check_user(user):
    formula = match({'pk': user['pk']})
    if(
        user['follower_count'] > THRESHOLD and \
        not table.first(formula=formula)
    ):
        for i in VOCAB:
            if user['full_name'].lower().find(i) != -1:
                return 0
        return 1
    return 0

In [ ]:
amount = 100
result = []
with open('hashtags', 'r') as f:
    print('---- start parsing -----')
    hashtags = f.read()
    for i in hashtags.splitlines():
        print(i.lower())
        result.append(np.array(cl.hashtag_medias_top(i.lower(), amount=amount)))

hash_tags_data = np.array(result)
hash_tags_data = hash_tags_data.reshape(-1)
print(hash_tags_data.shape)

for m, hashtag in enumerate(hash_tags_data):
    for n, post in enumerate(hashtag):
        user = cl.user_info(post.dict()['user']['pk']).dict()
        if check_user(user):
            user.pop('profile_pic_url')
            user.pop('profile_pic_url_hd')
    #         print(user)
            table.create(user)
        print('\rПрогресс {}/{}'.format(m*10+n, len(hash_tags_data)*len(hashtag)), end='')

In [157]:
to_score = table.all(view='Grid view', fields=['pk'])

In [206]:
def get_score(user_id):
    print(user_id, end='')
    media = cl.user_medias(user_id, amount=20)
    print(': ', end='')
    score = 0
    sample_data = set()
    morph = pymorphy2.MorphAnalyzer()

    for post in media:
        sample = post.caption_text
        for token in sample.split(' '):
#             print(token)
            if token not in stop_words and len(token) > 1:
#                 print(token)
#                 print(morph.parse(token)[0].normal_form)
                sample_data.update([morph.parse(token)[0].normal_form])
    
    score = len(set_core & sample_data)/len(sample_data)
    print(score)
    return score

In [208]:
scores = {}
for i in to_score:
    table.update(i['id'], {"score": get_score(i['fields']['pk'])})

1926936988: 0.0009587727708533077
12309131075: 0.002506265664160401
6198075009: 0.000945179584120983
3032662589: 0.003278688524590164
6074640286: 0.0
1994350047: 0.0007674597083653108
36734101231: 0.0
10423076968: 0.0012165450121654502
8078136902: 0.0
13857568912: 0.0
28182996526: 0.0006706908115358819
4520345844: 0.000819000819000819
4776437829: 0.0
21542888534: 0.0
864537599: 0.0
7294733170: 0.0
44718400882: 0.0
978069642: 0.0
11311432966: 0.0
5628166242: 0.001375515818431912
44439074768: 0.0
2201691392: 0.004842615012106538
2114984341: 0.0
9144866744: 0.0
5783740852: 0.0
39241982: 0.0020429009193054137
1940206804: 0.002677376171352075
265092080: 0.0014492753623188406
291123342: 0.0006583278472679394
45603813089: 0.0029154518950437317
1413751865: 0.0
1734476628: 0.0027100271002710027
2992751550: 0.001697792869269949
2045640885: 0.0
4988222132: 0.0
2225521139: 0.0006702412868632708
14449395045: 0.0
624910489: 0.001076426264800861
14916418338: 0.0
993496296: 0.001397624039133473
255788

In [201]:
len(set_core)

49